In [ ]:
def open_ecosystem_structure(site,date):
    '''Uses API call to retrieve NEON ecosystem structure (CHM)
    data at a given site and date. Returns XYZ. For more 
    information on NEON ecosystem structure data
    see https://data.neonscience.org/data-products/DP3.30015.001
    Parameters
    ----------
    site : str
        4 Letter site name. See 
        https://www.neonscience.org/field-sites/field-sites-map/list
        for a full list of NEON sites
    date : str
        Date of data collection in yyyy-mm format
    Returns
    -------
    
    '''
    data_product_url=['https://data.neonscience.org/api/v0/data/DP3.30015.001/'
                      +site+'/'+date]
    call_response = requests.get(data_product_url[0])
    call_response.json()
    CHM_raster_tiles=[]
    for i in call_response.json()['data']['files']:
        data_file_url=i['url']
        file_format=data_file_url.find('.tif')
        if not file_format == -1:
            CHM_raster_tiles.append(data_file_url)
    return CHM_raster_tiles

In [1]:
def open_NEON_presence_cover_plant(site,date):
    '''Uses API call to retrieve NEON product data
    for vegetation presence and coverage at a given 
    site and date. Returns pandas of 1 meter squared
    and 10&100 meter squared documents. For more 
    information on NEON presence cover plant data
    see https://data.neonscience.org/data-products/DP1.10058.001
    Parameters
    ----------
    site : str
        4 Letter site name. See 
        https://www.neonscience.org/field-sites/field-sites-map/list
        for a full list of NEON sites
    date : str
        Date of data collection in yyyy-mm format
    Returns
    -------
    small_subplot : pandas
        Pandas of the 1x1 meter subplots
    large_subplot : pandas 
        Pandas of the 10x10 and 100x100 meter
        subplots
    '''
    data_product_url=['https://data.neonscience.org/api/v0/data/DP1.10058.001/'
                      +site+'/'+date+'?package=expanded']
    call_response = requests.get(data_product_url[0])
    data_locations=[]
    for i in call_response.json()['data']['files']:
        data_file_url=i['url']
        file_format=data_file_url.find('m2Data')
        if not file_format == -1:
            data_locations.append(data_file_url)
    data_locations=sorted(data_locations)
    large_subplot = pd.read_csv(data_locations[0]).dropna(axis=1)
    small_subplot = pd.read_csv(data_locations[1]).dropna(axis=1)
    return small_subplot,large_subplot

In [ ]:
def open_woody_veg_structure(site,date):
    '''Uses API call to retrieve NEON product data
    for woody vegetation structure. Returns pandas 
    of merged apparent individual, mapping and tagging, 
    and per plot per year documents, eg one dataframe
    with locational, species, and height data. For more 
    information on NEON woody vegetation structure data products
    see https://data.neonscience.org/data-products/DP1.10098.001
    Parameters
    ----------
    site : str
        4 Letter site name. See 
        https://www.neonscience.org/field-sites/field-sites-map/list
        for a full list of NEON sites
    date : str
        Date of data collection in yyyy-mm format
    Returns
    -------
    small_subplot : pandas
        Pandas of the 1x1 meter subplots
    large_subplot : pandas 
        Pandas of the 10x10 and 100x100 meter
        subplots
    '''
    data_product_url=['https://data.neonscience.org/api/v0/data/DP1.10098.001/'
                      +site+'/'+date]
    call_response = requests.get(data_product_url[0])
    all_urls=[]
    for i in call_response.json()['data']['files']:
        data_file_url=i['url']
        height_find=data_file_url.find('individual')
        plot_find=data_file_url.find('perplot')
        map_find=data_file_url.find('mapping')
        if not height_find  == -1:
            apparent_df = pd.read_csv(data_file_url)
        elif not plot_find == -1:
            plot_df = pd.read_csv(data_file_url)
        elif not map_find ==-1:
            map_df = pd.read_csv(data_file_url)
    re_apparent_df=apparent_df[[
        'plotID','individualID','height']]
    re_plot_df=plot_df[['plotID',
                    'decimalLatitude','decimalLongitude',
                    'easting','northing']]
    re_map_df=map_df[['plotID','individualID','scientificName']]
    measurement_map_merge=pd.merge(
        re_apparent_df, re_map_df, on=['plotID','individualID'])
    all_merged_df=pd.merge(re_plot_df,measurement_map_merge,on='plotID')
    return all_merged_df


In [1]:
# def open_nonherb_veg_structure(site,date):
#     '''Uses API call to retrieve NEON product data
#     for woody vegetation structure Returns pandas of 1 meter squared
#     and 10&100 meter squared documents. For more 
#     information on NEON presence cover plant data
#     see https://data.neonscience.org/data-products/DP1.10045.001
#     Parameters
#     ----------
#     site : str
#         4 Letter site name. See 
#         https://www.neonscience.org/field-sites/field-sites-map/list
#         for a full list of NEON sites
#     date : str
#         Date of data collection in yyyy-mm format
#     Returns
#     -------
#     small_subplot : pandas
#         Pandas of the 1x1 meter subplots
#     large_subplot : pandas 
#         Pandas of the 10x10 and 100x100 meter
#         subplots
#     '''
#     data_product_url=['https://data.neonscience.org/api/v0/data/DP1.10045.001/'
#                       +site+'/'+date]
#     call_response = requests.get(data_product_url[0])
#     all_urls=[]
#     for i in call_response.json()['data']['files']:
#         data_file_url=i['url']
#         height_find=data_file_url.find('individual')
#         plot_find=data_file_url.find('perplot')
#         if not height_find  == -1:
#             apparent_df = pd.read_csv(data_file_url)
#         elif not plot_find == -1:
#             plot_df = pd.read_csv(data_file_url)
#     re_apparent_df=apparent_df[[
#         'plotID','individualID','height']]
#     re_plot_df=plot_df[['plotID',
#                     'decimalLatitude','decimalLongitude',
#                     'easting','northing']]
#     all_merged_df=pd.merge(re_plot_df,measurement_map_merge,on='plotID')
#     return all_merged_df